<a href="https://colab.research.google.com/github/vss121/Recommendation/blob/main/Item_Based_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Import related library
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [8]:
# 2. Import the dataframe
df_path = '/kaggle/input/df-rating/ratings.csv'

df = pd.read_csv(df_path)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/df-rating/ratings.csv'

In [12]:
# Colab에서만
# 2. Import the dataframe
from google.colab import drive
drive.mount('/content/drive')
file_path = "/content/drive/My Drive/ratings.csv"
df = pd.read_csv(file_path)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,userId,perfumeId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [10]:
# 3. Read in perfume_df
pf_path = '/kaggle/input/df-rating/perfumes.csv'
perfume_df = pd.read_csv(pf_path)
perfume_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/df-rating/perfumes.csv'

In [13]:
# Colab
# 3. Read in perfume_df
pf_path = '/content/drive/My Drive/perfumes.csv'
perfume_df = pd.read_csv(pf_path)
perfume_df.head()

,perfumeId,name,genres
0,1,name1,Adventure|Animation|Children|Comedy|Fantasy
1,2,name2,Adventure|Children|Fantasy
2,3,name3,Comedy|Romance
3,4,name4,Comedy|Drama|Romance
4,5,name5,Comedy


In [15]:
# 4. Merge 2 dataframe together
df_merge = pd.merge(df, perfume_df, on='perfumeId', how='left')
df_merge.drop(columns=['genres'], axis=1, inplace=True)
df_merge.head()

,userId,perfumeId,rating,timestamp,name
0,1,16,4.0,1217897793,name16
1,1,24,1.5,1217895807,name24
2,1,32,4.0,1217896246,name32
3,1,47,4.0,1217896556,name45
4,1,50,4.0,1217896523,name48


In [16]:
df.columns

Index(['userId', 'perfumeId', 'rating', 'timestamp'], dtype='object')

In [17]:
# 5. Df statistic
df = df_merge
df.describe()

,userId,perfumeId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [18]:
# 6. Create user-item interaction matrix
user_item_matrix = df.pivot_table(index='userId', columns='name', values='rating')
user_item_matrix.head()

name,name1,name10,name100,name1000,name10000,name10001,name10002,name10003,name10004,name10005,...,name9990,name9991,name9992,name9993,name9994,name9995,name9996,name9997,name9998,name9999
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# 7.1 Creating rating dataframe (mean rating of each movies)
ratings = pd.DataFrame(df.groupby('name')['rating'].mean())
ratings.head()

,rating
name,
name1,3.907328
name10,3.600000
name100,4.029661
name1000,3.984756
name10000,4.382353


In [20]:
# 7.2 New columns for the ratings df
ratings['no_rating'] = df.groupby('name')['rating'].count()
ratings.head()

,rating,no_rating
name,,
name1,3.907328,232
name10,3.600000,135
name100,4.029661,118
name1000,3.984756,164
name10000,4.382353,17


In [21]:
# 7.3 Popular movies (base on number of rating)
ratings.sort_values(by='no_rating', ascending=False).head(5)

,rating,no_rating
name,,
name261,4.160000,325
name317,4.138264,311
name280,4.454545,308
name427,3.659864,294
name526,4.194828,290


In [23]:
# 8.1 Making recommendation for the 'name261'
fg_df = user_item_matrix['name261']

In [24]:
# 8.2 Highest correlation with 'name261'
sim_fg = user_item_matrix.corrwith(fg_df)

In [25]:
# 8.3 Check the df
sim_fg.head()

name
name1        0.154139
name10       0.067621
name100      0.368278
name1000     0.095938
name10000    0.341432
dtype: float64

In [27]:
# 9. Create a final df
sim_fg = pd.DataFrame(sim_fg, columns=['Correlation'])
final_df = sim_fg.join(ratings['no_rating'])
final_df

,Correlation,no_rating
name,,
name1,0.154139,232
name10,0.067621,135
name100,0.368278,118
name1000,0.095938,164
name10000,0.341432,17
...,...,...
name9995,NaN,1
name9996,NaN,1
name9997,NaN,1


In [28]:
# 10. Specify the condition
final_df.loc[(final_df['no_rating']>30)].sort_values(by='Correlation', ascending=False)

,Correlation,no_rating
name,,
name261,1.000000,325
name9909,0.612300,32
name7494,0.578506,32
name1004,0.576556,40
name3874,0.571000,55
...,...,...
name708,-0.492456,37
name3383,-0.505274,45
name4706,-0.518541,45
